In [1]:
import os
import sys

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join("..", "src")))
import logging
import pandas as pd
import scanpy as sc
import decoupler as dc
from utils import load_config

# Load the configuration file
config = load_config("../config.yaml")


# Function to load gene expression data
def load_gene_expression_data(file_path, chunk_size=2000):
    """
    Load gene expression data in chunks to handle large file sizes.

    Args:
        file_path (str): Path to the preprocessed gene expression file.
        chunk_size (int): Size of chunks to load the data.

    Returns:
        pd.DataFrame: Combined gene expression matrix.
    """
    logging.info("Loading gene expression dataset in chunks...")
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        chunks.append(chunk)
    mat = pd.concat(chunks, axis=0)
    logging.info(f"Gene expression matrix loaded with shape: {mat.shape}")
    return mat


# # Load the gene expression matrix
# gene_df = load_gene_expression_data(config["data_paths"]["preprocessed_gene_file"])
# # Add an increasing integer ID column
# gene_df.insert(0, "ID", range(1, len(gene_df) + 1))


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\20191678\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\traitlets\con

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\20191678\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\traitlets\con

AttributeError: _ARRAY_API not found

In [ ]:
# Extract perturbation IDs and drop target/metadata columns
pert_ids = gene_df["ID"]
mat = gene_df.drop(columns=["ID", "viability", "cell_mfc_name", "pert_dose"])

# Create AnnData object for Decoupler
adata = sc.AnnData(
    X=mat.values, obs=pd.DataFrame(index=pert_ids), var=pd.DataFrame(index=mat.columns)
)
print(f"AnnData object created with shape: {adata.shape}")

In [ ]:
# Load CollecTRI network
net = dc.get_collectri(organism="human", split_complexes=False)

# Verify network format
print("Regulatory Network (First Rows):")
print(net.head())

# Check alignment between network targets and gene expression
shared_genes = set(net["target"]).intersection(adata.var_names)
print(f"Number of shared genes: {len(shared_genes)}")
assert (
    len(shared_genes) > 0
), "No shared genes between network and gene expression matrix!"

# Filter the network for matching genes
net_filtered = net[net["target"].isin(shared_genes)]
print(f"Filtered network has {len(net_filtered)} interactions.")

In [ ]:
# Run TF activity inference with ULM
dc.run_ulm(
    mat=adata,
    net=net_filtered,
    source="source",
    target="target",
    weight="weight",
    verbose=True,
    min_n=2,  # Minimum targets per TF
    use_raw=False,  # Use raw gene expression data
)

# Store the results for downstream analysis
print("ULM Results (TF Activities):")
print(adata.obsm["ulm_estimate"].head())

In [ ]:
dc.plot_network(
    net=net,
    n_sources=["PAX5", "EBF1", "RFXAP"],
    n_targets=15,
    node_size=100,
    s_cmap="white",
    t_cmap="white",
    c_pos_w="darkgreen",
    c_neg_w="darkred",
    figsize=(5, 5),
)

In [2]:
net = dc.get_collectri(organism="human", split_complexes=False)
net

  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\connectionpool.py", line 466, in _make_request
    self._validate_conn(conn)
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\connectionpool.py", line 1095, in _validate_conn
    conn.connect()
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\connection.py", line 730, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\connection.py", line 909, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\util\ssl_.py", line 469, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(sock, context, tls_in_tls, server_hostname)
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\u

AttributeError: module 'numpy' has no attribute 'NAN'

In [4]:
net = dc.get_collectri(organism="human", split_complexes=False)
net

AttributeError: module 'numpy' has no attribute 'NAN'